In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
stocks = pd.read_csv("./input/sp500_stocks.csv")

In [25]:
stocks

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2009-12-31,MMM,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-04,MMM,59.318886,83.019997,83.449997,82.669998,83.089996,3043700.0
2,2010-01-05,MMM,58.947342,82.500000,83.230003,81.699997,82.800003,2847000.0
3,2010-01-06,MMM,59.783295,83.669998,84.599998,83.510002,83.879997,5268500.0
4,2010-01-07,MMM,59.826176,83.730003,83.760002,82.120003,83.320000,4470100.0
...,...,...,...,...,...,...,...,...
1607583,2022-08-29,ZTS,157.869995,157.869995,159.139999,157.529999,158.309998,1236900.0
1607584,2022-08-30,ZTS,157.080002,157.080002,158.750000,156.759995,158.479996,1918800.0
1607585,2022-08-31,ZTS,156.529999,156.529999,159.410004,156.210007,158.160004,2149000.0
1607586,2022-09-01,ZTS,159.410004,159.410004,159.750000,154.720001,155.789993,1839400.0


In [26]:
# GOOGL
# AAPL
# stock1 = stocks[stocks[""]]
stock1 = stocks[stocks["Symbol"]=="GOOGL"].dropna(subset=["Close"])
stock2 = stocks[stocks["Symbol"]=="AAPL"].dropna(subset=["Close"])
dates = pd.merge(stock1[["Date"]],stock2[["Date"]])["Date"]

In [27]:
stock1 = stock1[stock1["Date"].isin(dates)]
stock1["LogReturn"] = np.log(stock1["Close"] / stock1["Close"].shift(+1))
name = stock1["Symbol"].unique()[0]
stock1[["Date", "Symbol", "LogReturn"]].dropna().to_csv(f"./input/stocks/{name}.csv", index=False)

In [28]:
stock2 = stock2[stock2["Date"].isin(dates)]
stock2["LogReturn"] = np.log(stock2["Close"] / stock2["Close"].shift(+1))
name = stock2["Symbol"].unique()[0]
stock2[["Date", "Symbol", "LogReturn"]].dropna().to_csv(f"./input/stocks/{name}.csv", index=False)

In [29]:
df = pd.read_csv("./input/sp500_index.csv")
df = df[df["Date"].isin(dates)]
df["LogReturn"] = np.log(df["S&P500"] / df["S&P500"].shift(+1))
name = "INDEX"
df["Symbol"] = name
df[["Date","Symbol","LogReturn"]].dropna().to_csv(f"./input/stocks/{name}.csv", index=False)

In [30]:
def convert(df, symbol):
    stock = stocks[stocks["Symbol"]==symbol].dropna(subset=["Close"])
    stock["LogReturn"] = np.log(stock["Close"] / stock["Close"].shift(+1))
    name = stock["Symbol"].unique()[0]
    stock[["Date", "Symbol", "LogReturn"]].dropna().to_csv(f"./input/stocks/{name}.csv", index=False)

In [31]:
stks = stocks.groupby("Symbol").mean()["Volume"].sort_values(ascending=False).head(20).index
for stk in stks:
    convert(stocks, stk)

In [32]:
import yaml
d_out = {v:i+1 for i,v in enumerate(stks)}
d_out["INDEX"] = 0
with open("./src/utils/mapping.yml", "wt") as F:
    yaml.safe_dump(d_out, F)

In [33]:
import glob
# df1 = pd.read_csv("./input/stocks/AAPL.csv")
# df2 = pd.read_csv("./input/stocks/GOOGL.csv")
# df3 = pd.read_csv("./input/stocks/INDEX.csv")
files = glob.glob("./input/stocks/*.csv")
df = pd.concat([pd.read_csv(file) for file in files])
data = df.pivot(index="Date", columns="Symbol", values="LogReturn").dropna()
data.index = pd.to_datetime(data.index)
# data = data.asfreq("1D")
# data = data[data.index.dayofweek.isin([0,1,2,3,4])]
# data = data.ffill()
st = pd.Timestamp(year=2012,month=9,day=5)
ed = st + pd.Timedelta(days=1) * 2515

data.index = pd.date_range(start=st,end=ed)
data.to_csv("pseudodata.csv")